# Integral de Linha de Campos Vetoriais

## Preamble

In [1]:
import av_utils as av
import numpy as np
import sympy as sp
import sympy.vector as sv
import plotly.graph_objects as go
from sympy.plotting.plot import plot3d, plot3d_parametric_line, plot3d_parametric_surface, plot_parametric
import matplotlib.pyplot as plt

Preparandndo o ambiente de plotagem

In [2]:
sp.init_printing()

Definir algumas variaveis para usar

In [3]:
x,y,z,t,s,u,v, phi, theta = sp.symbols('x y z t s u v phi theta',real=True)

Definir um sistema de coordenadas

In [4]:
R = sv.CoordSys3D('R')

## Introdução

Desta vez, estamos olhando para algo que é relacionado, mas distinto do chapter anterior: integrais de linha de campos vetoriais. Você deve separar claramente esses dois conceitos em sua mente, e pergunte a si mesmo, cada vez que for calcular uma integral de linha, "Que tipo de função estou integrando?" Seu livro didático geralmente ajuda escrevendo integrais de linha de funções escalares com o diferencial $ds$, e integrais de linha de campos vetoriais com o diferencial $dr$ ou $d\vec r$ Útil, não é? 
Por outro lado, você não deve se deixar confundir pelos símbolos $\int_C$ e $\oint_C$ - eles significam quase a mesma coisa! O símbolo $\int_C$ sempre pode ser usado no lugar de $\oint_C$. Escrevemos $\oint_C$ quando queremos um lembrete extra de que C é um caminho fechado (termina no mesmo ponto em que começou). É claro que não escrevemos $\oint_C$ se C não está fechado!
Finalmente, esteja avisado que, em alguns livros, "integral de linha" e "integral de caminho" são sinônimos, enquanto outros livros os chamam de "integrais de linha de campos de vetores" e integrais de linha de funções escalares", respetivamente.
$$
\int_C \vec F \cdot d\vec r = \int_a^b \vec F (\vec r(t))\cdot \vec{r^\prime}(t) dt.
$$
Tem outra forma de escrever essa integral de linha expressando os vetores em termo dos seus componentes. $d\vec r = dx \hat i + dy \hat j$, e $\vec F(x,y) = P(x,y)\hat i + Q(x,y) \hat j$. Nesta forma podemos reescrever a integral na seguinte maneira 
$$
\begin{align}
\int_C \vec F \cdot d\vec r &= \int_C(P(x,y)\hat i + Q(x,y) \hat j)\cdot(dx \hat i + dy \hat j) \\ 
&=\int_C P(x,y) dx + Q(x,y) dy.
\end{align}
$$
A parametrização da integral acima é dada por
$$
\int_C P(x,y) dx + Q(x,y) dy = \int_a^b P(x(t),y(t)) x^\prime(t) dt + Q(x(t),y(t)) y^\prime(t)dt.
$$

**Exemplo:**
Calcule a integral $ \int_C \vec F \cdot d\vec r$ onde $\vec F(x,y,z) = (z^3,y^2,x)$ e $C$ é a curva parametrizada $\vec r(t) = (t,\sin(t),1), \;0\leq t \leq \pi$ 

**Resolução:**
Primeiro vamos resolver sem computador e depois implementamos no Python.
* 1-
O primeiro passo é determinar a equação parametrica da curva. Nesta questão, a equação é dada. O segundo passo é escrever a equação do campo vetorial que tambem é dada nesta questão. Agora temos que parametrizar o campo vetorial usando a equação da curva. 
$$
\vec F(x,y,z) \rightarrow \vec F(x(t),y(t),z(t)) = (1^3, \sin^2(t),t).
$$
O proximo passo é calcular $d\vec r$,
$$
d\vec r = x^\prime(t) dt \hat i + y^\prime(t) dt \hat j+ z^\prime(t) dt \hat k= 1 dt \hat i + \cos(t) dt \hat j + 0 dt \hat k.
$$
Agora temos todos os ingredientes prontos para resolver a integral,

$$
\begin{align}
\int_C \vec F \cdot d\vec r &= \int^\pi_0 (1^3, \sin^2(t),t^3) \cdot (1 dt \hat i + \cos(t) dt \hat j + 0 dt \hat k)\\
&= \int^\pi_0 dt + \cos(t)\sin^2(t) dt + 0 \\
&= t \big\rvert^\pi_0 + \frac{\sin^3(t)}{3}\big\rvert^\pi_0 =\pi.
\end{align}
$$
* 2- 
Agora vamos implementar esse procedimento no Python. Obviamente, primeiro temos que definir a curva e o campo como uma função. 

In [5]:
def curve1(t): return t*R.i + sp.sin(t)*R.j + R.k

In [6]:
curve1(t)

In [7]:
def field1(x,y,z): return z**3*R.i + y**2*R.j + x*R.k

In [8]:
field1(x,y,z)

Antes de começar a calcular a integral vamos dar uma olhada no grafico da curva e do compo

In [20]:
fig = av.plot3d_vector_field(field1(x,y,z),(x,-2,2),(y,-2,2),(z,-2,2),points=5)

In [21]:
fig

In [22]:
av.plot3d_parametric_curve(curve1(t),(t,-np.pi,2*np.pi), fig=fig)

Parametrizando o campo,

In [11]:
paramet_field = field1(*curve1(t).components.values())

In [12]:
paramet_field

In [13]:
dr = curve1(t).diff()

In [14]:
dr

O integrando é o produto escalar entre campo vetorial parametrizado e elemento da linha ($d\vec r$) 

In [15]:
integrand = paramet_field.dot(dr)

In [16]:
integrand

Finalmente podemos integrar 

In [17]:
sp.integrate(integrand,(t,0,sp.pi))

Para facilitar a vida, vamos montar uma função que faça o procedimento acima automaticamente

In [18]:
def line_integral_vectorial(field,curve,a):
    '''
    - Arguments:
        `field`: Vector field F(x,y,z) = P(x,y,z)i + R(x,y,z)j + Q(x,y,z)k. The parameters of the field must be `x`,`y` and `z`
        `curve`: parametrized curve r(t) = x(t)i + y(t)j + z(t)k
        `a`:(Tuple) (parameter of the curve, initial point, final point)
        **Note**: the dimensionality of curve and field must be the same. 
    - Return:
        line integral of the vectorial filed along the curve for the given interval. 


    Example:
    =====
    import sympy as sp
    import sympy.vector as sv
    R = sv.CoordSys3D('R')
    t,x,y,z = sp.symbols('t x y z')
    def f(x,y,z):
        x*R.i + y*z*R.j + x*z*R.k
    def r(t):
        return 2*t*R.i + t**2*R.j - R.k
    
    line_integral_vectorial(f(x,y,z), r(t), (t,-1,2))
    
    '''

    #taking the name of the coordinate system. Here we assume that the filed and curve are using the same coordinate system
    R = list(curve.separate().keys())[0]
    R_f = list(field.separate().keys())[0]

    assert R==R_f, 'the given filed and curve must be in a same coordinate system'
    
    x_c  = curve.dot(R.i)
    y_c  = curve.dot(R.j)
    z_c  = curve.dot(R.k)
    
    param = [p for p in curve.free_symbols if not p.is_Vector]
    assert len(param)==1, "A curve has only one parameter"
    assert param[0]==a[0], "the parameter of the curve must be the same as the integration variable."

    # parametrizing the field using the curve equation
    parametrized_field =field.subs(x, x_c).subs(y, y_c).subs(z, z_c)
    
    
    integrand = parametrized_field.dot(curve.diff())
    
        
    return sp.integrate(integrand,a).evalf()

In [19]:
line_integral_vectorial(field1(x,y,z),curve1(t),(t,0,sp.pi))